In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import matplotlib.pylab as pylab
import matplotlib as mpl
import cartopy.feature as cfeature
import scipy.stats as stats
import xarray as xr
import ipdb
import glob
import itertools
import numpy.ma as ma
from utils import u_statistics as u_stat
from scipy.stats import gaussian_kde, linregress
import pickle as pkl
from utils import u_plot as uplot, u_darrays as uda, u_met
import pandas as pd
import salem
import shapely.geometry as shpg
from matplotlib.colors import LinearSegmentedColormap

##for regridding, install xesmf:
# conda install esmpy
# pip install xesmf
from utils import constants as cnst

pylab.rcParams['figure.figsize'] = (30., 30.)
#%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [ ]:
mcs50 = '/home/ck/DIR/mymachine/GRIDSAT/MCS18/gridsat_WA_-50_2010.nc'
mcs70 = '/home/ck/DIR/mymachine/GRIDSAT/MCS18/gridsat_WA_2010.nc'
era = cnst.local_data + 'ERA5/hourly/surface/ERA5_2010_04_srfc.nc'
topo = cnst.WA_TOPO_1MIN

In [ ]:
d50 = xr.open_dataarray(mcs50).sel(lon=slice(-18,20), lat=slice(2,23)).sel(time='2010-04-22')
d70 = xr.open_dataarray(mcs70).sel(lon=slice(-18,20), lat=slice(2,23)).sel(time='2010-04-22')
era5 = xr.open_dataset(era)
era5 = uda.flip_lat(era5)
era5 = era5['t2m'].sel(longitude=slice(-18,20), latitude=slice(2,23)).sel(time='2010-04-22')

In [ ]:
top = xr.open_dataarray(topo)

In [ ]:
era5p = era5[era5['time.hour']==12]-273.15

In [ ]:
era5p

In [ ]:
#d70.values[d70.values>-70] = np.nan
d50.values[d50.values>-50] = np.nan
lon,lat = np.meshgrid(d70.lon, d70.lat)


In [ ]:
lats = lat[d70.squeeze().values<=-70]
lons = lon[d70.squeeze().values<=-70]

In [ ]:
fp = ''
cmap = mpl.cm.viridis(np.linspace(0,0.9,6))
cmap = mpl.colors.ListedColormap(cmap)

map = d50.salem.get_map()
x, y = map.grid.transform(lons, lats)

f = plt.figure(figsize=(10,5), dpi=300)

ax = f.add_subplot(111)

#map.set_rgb(natural_earth='lr')
map.set_plot_params(levels=[-80,-70,-60,-50], cmap=cmap, extend='both')

map.set_data(d50)  # interp='linear'

map.set_contour((d70.values).astype(np.float64), interp='linear', colors='yellow', linewidths=1, levels=[-70,-65])
#   # levels=np.arange(-0.5,0.51,0.1),

# map.set_contour(era5p, interp='linear', colors='red', linewidths=1, levels=[35,50])
#   # levels=np.arange(-0.5,0.51,0.1),

coord = [-12, 12, 4.5, 8.5]
geom = shpg.box(coord[0], coord[2], coord[1], coord[3])
map.set_geometry(geom, zorder=99, edgecolor='orange', linewidth=3,  alpha=0.3, facecolor='none')

coord = [-12, 12, 9, 20]
geom = shpg.box(coord[0], coord[2], coord[1], coord[3])
map.set_geometry(geom, zorder=99, edgecolor='blue', linewidth=3,  alpha=0.3, facecolor='none')

# coord = [-10, 20, 9, 20]
# geom = shpg.box(coord[0], coord[2], coord[1], coord[3])
# map.set_geometry(geom, zorder=99, edgecolor='purple', linewidth=3,  alpha=0.3, facecolor='none')
map.set_shapefile(oceans=True)
#map.set_shapefile(rivers=True)
#map.set_topography(top,relief_factor=0.7)

#plt.text(70,220,'June-September', fontsize=11)
#plt.scatter(x, y, facecolor='yellow', edgecolors='yellow', zorder=99, s=0.15)
dic = map.visualize(ax=ax, title='22-04-2010 18:00UTC', cbar_title='C$^{\circ}$')

# contours = dic['contour'][0]
# plt.clabel(contours, inline=True, fontsize=7, fmt='%1.1f')

plt.tight_layout()
plt.savefig(cnst.network_data + 'figs/CLOVER/fig0.jpg')

In [3]:
era_srfc = xr.open_dataset(cnst.ERA5_MONTHLY_SRFC_SYNOP)
era_pl = xr.open_dataset(cnst.ERA5_MONTHLY_PL_SYNOP)
ccover = xr.open_dataset(cnst.GRIDSAT + 'aggs/gridsat_WA_-70_monthly_mean_5000km2.nc')
#chirps = xr.open_mfdataset(cnst.CHIRPS + "*.nc")

In [4]:
era_srfc = era_srfc.where((era_srfc['t2m'])['time.hour'] == 12, drop=True)
era_pl = era_pl.where((era_pl['q'])['time.hour'] == 12, drop=True)
era_srfc = u_darrays.flip_lat(era_srfc)
era_pl = u_darrays.flip_lat(era_pl)

MemoryError: 

In [ ]:
test = era_pl.sel(longitude=slice(-12,10), latitude=slice(5,8)).sel(level=slice(700,750)).mean(dim='level')
test1 = era_pl.sel(longitude=slice(-12,10), latitude=slice(5,8)).sel(level=slice(900,950)).mean(dim='level')
test3 = era_pl.sel(longitude=slice(-12,10), latitude=slice(5,8)).sel(level=slice(600,650)).mean(dim='level')
test2 = era_srfc.sel(longitude=slice(-12,10), latitude=slice(5,8))

In [ ]:
mean = test.mean(['longitude']).groupby('time.month').max('latitude').groupby('time.month').mean('time')
mean1 = test1.mean(['longitude']).groupby('time.month').max('latitude').groupby('time.month').mean('time')
mean2 = test2.mean(['longitude']).groupby('time.month').max('latitude').groupby('time.month').mean('time')
mean3 = test3.mean(['longitude']).groupby('time.month').max('latitude').groupby('time.month').mean('time')

In [ ]:
cmean_coll = ccover['tir'].sel(lat=slice(5,8), lon=slice(-12,10)).mean(['lon']).groupby('time.month').max('lat').groupby('time.month').mean('time')